In [111]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/project/spark-3.2.1-bin-hadoop3.2"

In [112]:
import glob
import json

In [113]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark App") \
    .config("spark.jars", "/project/postgresql-42.3.2.jar") \
    .getOrCreate()

In [127]:
postgres_uri = "jdbc:postgresql://demsin0166.czfwea5noxbs.eu-west-2.rds.amazonaws.com:5432/initialdatabase"
dbtable = "public.google"
user = "luan"
password = "qwerty1234"

In [115]:
df = spark.read \
    .format("jdbc") \
    .option("url", postgres_uri) \
    .option("dbtable", dbtable) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.printSchema()

root
 |-- google_id: string (nullable = true)
 |-- restaurant_id: string (nullable = true)
 |-- restaurant_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- website: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- total_ratings: string (nullable = true)
 |-- price_level: string (nullable = true)



In [94]:
def get_variables(data):
    google_id = data['google_id']
    identifier = data['result']['name'].replace(" ", "").replace("'", "").replace("’", "").lower()
    restaurant_name = data['result']['name'].replace("'", "").replace("’", "")
    try:
        address = data['result']['formatted_address'].replace("'", "")
    except:
        address = 'no address'
    try:
        postcode = data['result']['address_components'][6]['long_name']
    except:
        postcode = 'no postcode'
    try:
        website = data['result']['website']
    except:
        website = 'no website'
    try:
        phone_number = data['result']['formatted_phone_number']
    except:
        phone_number = 'no phone number'
    try:
        opening_hours = str(data['result']['opening_hours']['weekday_text'])
    except:
        opening_hours = 'no opening hours'
    try:
        rating = str(data['result']['rating'])
    except:
        rating = 'no rating'
    try:
        total_ratings = str(data['result']['user_ratings_total'])
    except:
        total_ratings = 'no total ratings'
    try:
        price_level = str(data['result']['price_level'])
    except:
        price_level = 'no price level'
    
    
    return google_id, identifier,restaurant_name, address, postcode, website, phone_number, rating, total_ratings, price_level

In [126]:
parquet_filepath = []

for filepath in glob.iglob('/project/parquet/*.parquet'):
    parquet_filepath.append(filepath)

In [68]:
def convert_json(restaurant):
    df1 = spark.read.parquet(restaurant)
    json_data = json.loads(df1.toJSON().first())
    return get_variables(json_data)

In [96]:
convert_json('/project/parquet/Amrutha Lounge.parquet')

('ChIJiSGXNhYGdkgRnpnw1YxNrqo',
 'amruthalounge',
 'Amrutha Lounge',
 '326 Garratt Ln, London SW18 4EJ, UK',
 'SW18 4EJ',
 'http://www.amrutha.co.uk/',
 '020 8001 4628',
 '4.9',
 '884',
 '1')

In [95]:
df2 = [convert_json(i) for i in parquet_filepath]

In [124]:
google_df = spark.createDataFrame(
        df2,
    ['google_id', 'restaurant_id', 'restaurant_name', 'address', 'postcode', 'website', 'phone_number', 'rating','total_ratings', 'price_level']
)

In [123]:
google_df.write.mode("append") \
    .format("jdbc") \
    .option("url", postgres_uri) \
    .option("dbtable", "public.google") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .save()